In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader

In [3]:
import pandas as pd
import torch
from transformers import BertForTokenClassification,PretrainedConfig,AutoConfig
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import StepLR
from torch.nn.functional import softmax
from sklearn.model_selection import KFold
import random
import os
from sklearn.metrics import f1_score

In [4]:
device= torch.device("cuda")
device

device(type='cuda')

In [5]:
def transform_df(df):
    df['label'] = df['label'].apply(lambda x: [0, 1] if x == 1 else [1, 0])
    return df

In [6]:
data = pd.read_json("/kaggle/input/meme-dataset/train.jsonl",lines=True)
test = pd.read_json("/kaggle/input/meme-dataset/test.jsonl",lines=True)
val = pd.read_json("/kaggle/input/meme-dataset/dev.jsonl",lines=True)
val.to_csv('predictions.tsv', index = False, sep = '\t')

In [7]:
data = transform_df(data)
val = transform_df(val)

In [8]:
data.head()

,id,img,label,text
0,42953,img/42953.png,"[1, 0]",its their character not their color that matters
1,23058,img/23058.png,"[1, 0]",don't be afraid to love again everyone is not ...
2,13894,img/13894.png,"[1, 0]",putting bows on your pet
3,37408,img/37408.png,"[1, 0]",i love everything and everybody! except for sq...
4,82403,img/82403.png,"[1, 0]","everybody loves chocolate chip cookies, even h..."


In [9]:
test.head()

,id,img,text
0,16395,img/16395.png,handjobs sold seperately
1,37405,img/37405.png,introducing fidget spinner for women
2,94180,img/94180.png,happy pride month let's go beat up lesbians
3,54321,img/54321.png,laughs in [majority of u.s crime rate]
4,97015,img/97015.png,finds out those 72 virgins.. are goats


In [10]:
val.head()

,id,img,label,text
0,8291,img/08291.png,"[0, 1]",white people is this a shooting range
1,46971,img/46971.png,"[0, 1]",bravery at its finest
2,3745,img/03745.png,"[0, 1]",your order comes to $37.50 and your white priv...
3,83745,img/83745.png,"[0, 1]",it is time.. to send these parasites back to t...
4,80243,img/80243.png,"[0, 1]",mississippi wind chime


In [11]:
!pip install pytorch-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.4/176.4 kB 1.6 MB/s eta 0:00:00a 0:00:01
  Obtaining dependency information for sacremoses from https://files.pythonhosted.org/packages/0b/f0/89ee2bc9da434bd78464f288fdb346bc2932f2ee80a90b2a4bbbac262c74/sacremoses-0.1.1-py3-none-any.whl.metadata
  Obtaining dependency information for botocore<1.30.0,>=1.29.100 from https://files.pythonhosted.org/packages/46/20/e7a9a8e6746872afcc4e3ad5ab503702c38813b3a532df27cce95c98b8cb/botocore-1.29.165-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 10.8 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 37.9 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: botocore
    Found existing installation: botocore 1.33.1
    Uninstalling botocore-1.33.1:
      Successfully uninstalled botocore-1.33.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of th

In [12]:
from pytorch_transformers import RobertaModel, RobertaTokenizer
from pytorch_transformers import RobertaForSequenceClassification, RobertaConfig

In [13]:
class PairsDataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __getitem__(self, idx):
        return (self.x[idx], self.y[idx])

    def __len__(self):
        return len(self.x)
    
def data_collator(batch):
    y = torch.Tensor([p[1] for p in batch]).to(device)
    x = tokenizer([p[0] for p in batch], return_tensors='pt', padding='max_length',truncation=True).to(device)
    return (x, y)

In [14]:
def get_dataloaders(train, valid):
    train_dataset = PairsDataset(train.text.values, train.label.values)
    train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, drop_last=False, shuffle=True, collate_fn=data_collator)
    
    valid_dataset = PairsDataset(valid.text.values, valid.label.values)
    valid_dataloader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, drop_last=False, shuffle=False, collate_fn=data_collator)
    
    return train_dataloader, valid_dataloader

In [15]:
import sklearn
def evaluate_model(model, test_dataloader):
    num = 0
    den = 0
    y_true = list()
    y_pred = list()
    y_pred_prob = list()
    f1_valid = .0
    for x, y in test_dataloader:
        with torch.no_grad():
            output = model(
                input_ids=x.input_ids,
                attention_mask=x.attention_mask,
                labels=y,
                return_dict=True
            )
            loss = output.loss
            
            num += len(x) * loss.item()
            den += len(x)
            
            y_pred.extend(torch.argmax(output.logits, 1).tolist())
            y_pred_prob.extend(softmax(output.logits, dim = 1)[:, 1].tolist())
            y_true.extend(torch.argmax(y, 1).tolist())
            
    val_loss = num / den
    acc = sklearn.metrics.accuracy_score(y_true, y_pred)
    return val_loss, acc, y_pred_prob

In [16]:
def train_loop(
    model, train_dataloader, val_dataloader, 
    max_epochs=10, 
    lr=1e-5,
    eval_steps = 50
):
    optimizer = torch.optim.Adam(params = model.parameters(), lr=lr)
    scheduler = StepLR(optimizer, step_size = 3, gamma=0.5)
    best_acc = float('-inf')
    
    for epoch in range(max_epochs):
        print('EPOCH', epoch)
        losses = list()
        for i, (x, y) in enumerate(train_dataloader):            
            output = model(
                input_ids=x.input_ids,
                attention_mask=x.attention_mask,
                labels=y,
                return_dict=True
            )
            loss = output.loss
            
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            
            losses.append(loss.item())
            
            if i % eval_steps == 0:
                model.eval()
                train_loss = np.mean(losses[-eval_steps:])
                eval_loss, eval_acc, _ = evaluate_model(model, val_dataloader)
                if eval_acc > best_acc:
                    best_acc = eval_acc
                    torch.save(model.state_dict(), SAVE_PATH)
                print(f'step {i} train_loss: {train_loss:.3} eval_loss: {eval_loss:.3} eval_acc: {eval_acc:.3}')
                model.train()
        scheduler.step()

In [17]:
from sklearn.model_selection import KFold

In [18]:
""" CharacterTokenzier for Hugging Face Transformers.

This is heavily inspired from CanineTokenizer in transformers package.
"""
import json
import os
from pathlib import Path
from typing import Dict, List, Optional, Sequence, Union

from transformers.tokenization_utils import AddedToken, PreTrainedTokenizer


class CharacterTokenizer(PreTrainedTokenizer):
    def __init__(self, characters: Sequence[str], model_max_length: int, **kwargs):
        """Character tokenizer for Hugging Face transformers.

        Args:
            characters (Sequence[str]): List of desired characters. Any character which
                is not included in this list will be replaced by a special token called
                [UNK] with id=6. Following are list of all of the special tokens with
                their corresponding ids:
                    "[CLS]": 0
                    "[SEP]": 1
                    "[BOS]": 2
                    "[MASK]": 3
                    "[PAD]": 4
                    "[RESERVED]": 5
                    "[UNK]": 6
                an id (starting at 7) will be assigned to each character.

            model_max_length (int): Model maximum sequence length.
        """
        self.characters = characters
        self.model_max_length = model_max_length
        bos_token = AddedToken("[BOS]", lstrip=False, rstrip=False)
        eos_token = AddedToken("[SEP]", lstrip=False, rstrip=False)
        sep_token = AddedToken("[SEP]", lstrip=False, rstrip=False)
        cls_token = AddedToken("[CLS]", lstrip=False, rstrip=False)
        pad_token = AddedToken("[PAD]", lstrip=False, rstrip=False)
        unk_token = AddedToken("[UNK]", lstrip=False, rstrip=False)

        mask_token = AddedToken("[MASK]", lstrip=True, rstrip=False)
        self._vocab_str_to_int = {
            "[CLS]": 0,
            "[SEP]": 1,
            "[BOS]": 2,
            "[MASK]": 3,
            "[PAD]": 4,
            "[RESERVED]": 5,
            "[UNK]": 6,
            **{ch: i + 7 for i, ch in enumerate(characters)},
        }
        super().__init__(
            bos_token=bos_token,
            eos_token=eos_token,
            sep_token=sep_token,
            cls_token=cls_token,
            pad_token=pad_token,
            mask_token=mask_token,
            unk_token=unk_token,
            add_prefix_space=False,
            model_max_length=model_max_length,
            **kwargs,
        )

        self._vocab_int_to_str = {v: k for k, v in self._vocab_str_to_int.items()}

    @property
    def vocab_size(self) -> int:
        return len(self._vocab_str_to_int)

    def _tokenize(self, text: str) -> List[str]:
        return list(text)

    def _convert_token_to_id(self, token: str) -> int:
        return self._vocab_str_to_int.get(token, self._vocab_str_to_int["[UNK]"])

    def _convert_id_to_token(self, index: int) -> str:
        return self._vocab_int_to_str[index]

    def convert_tokens_to_string(self, tokens):
        return "".join(tokens)

    def build_inputs_with_special_tokens(
        self, token_ids_0: List[int], token_ids_1: Optional[List[int]] = None
    ) -> List[int]:
        sep = [self.sep_token_id]
        cls = [self.cls_token_id]
        result = cls + token_ids_0 + sep
        if token_ids_1 is not None:
            result += token_ids_1 + sep
        return result

    def get_special_tokens_mask(
        self,
        token_ids_0: List[int],
        token_ids_1: Optional[List[int]] = None,
        already_has_special_tokens: bool = False,
    ) -> List[int]:
        if already_has_special_tokens:
            return super().get_special_tokens_mask(
                token_ids_0=token_ids_0,
                token_ids_1=token_ids_1,
                already_has_special_tokens=True,
            )

        result = [1] + ([0] * len(token_ids_0)) + [1]
        if token_ids_1 is not None:
            result += ([0] * len(token_ids_1)) + [1]
        return result

    def create_token_type_ids_from_sequences(
        self, token_ids_0: List[int], token_ids_1: Optional[List[int]] = None
    ) -> List[int]:
        sep = [self.sep_token_id]
        cls = [self.cls_token_id]

        result = len(cls + token_ids_0 + sep) * [0]
        if token_ids_1 is not None:
            result += len(token_ids_1 + sep) * [1]
        return result

    def get_config(self) -> Dict:
        return {
            "char_ords": [ord(ch) for ch in self.characters],
            "model_max_length": self.model_max_length,
        }
    
    def get_vocab(self):
        return self._vocab_str_to_int
    
    @classmethod
    def from_config(cls, config: Dict) -> "CharacterTokenizer":
        cfg = {}
        cfg["characters"] = [chr(i) for i in config["char_ords"]]
        cfg["model_max_length"] = config["model_max_length"]
        return cls(**cfg)

    def save_pretrained(self, save_directory: Union[str, os.PathLike], **kwargs):
        cfg_file = Path(save_directory) / "tokenizer_config.json"
        cfg = self.get_config()
        with open(cfg_file, "w") as f:
            json.dump(cfg, f, indent=4)

    @classmethod
    def from_pretrained(cls, save_directory: Union[str, os.PathLike], **kwargs):
        cfg_file = Path(save_directory) / "tokenizer_config.json"
        with open(cfg_file) as f:
            cfg = json.load(f)
        return cls.from_config(cfg)

In [19]:
import string
import sys

chars = "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz"
model_max_length = 64
tokenizer = CharacterTokenizer(chars, model_max_length)

In [20]:
from transformers import DebertaV2Config

# config = DebertaV2Config(
#     vocab_size=74,  # размер словаря
#     hidden_size=128,  # размер скрытого состояния
#     num_hidden_layers=4,  # количество слоев
#     num_attention_heads=8,  # количество внимательных голов
#     intermediate_size=1024,  # размер промежуточного слоя
#     hidden_act="gelu",  # функция активации
#     hidden_dropout_prob=0.2,  # вероятность отключения для Dropout в скрытом слое
#     attention_probs_dropout_prob=0.2,  # вероятность отключения для Dropout внимательных голов
#     max_position_embeddings=64,
#     max_length= 66,
# #max_relative_positions=64,
#     type_vocab_size=0,  # размер словаря типов
#     pooler_dropout= 0,
#     pooler_hidden_act="gelu",
#    pooler_hidden_size= 128,
#    position_biased_input = True,
#    relative_attention = True,
#    initializer_range=0.02,
#     num_classes=2,
# )
config = DebertaV2Config.from_pretrained("microsoft/deberta-v3-base")
config.num_labels = 2
config

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

DebertaV2Config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-07,
  "max_position_embeddings": 512,
  "max_relative_positions": -1,
  "model_type": "deberta-v2",
  "norm_rel_ebd": "layer_norm",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_dropout": 0,
  "pooler_hidden_act": "gelu",
  "pooler_hidden_size": 768,
  "pos_att_type": [
    "p2c",
    "c2p"
  ],
  "position_biased_input": false,
  "position_buckets": 256,
  "relative_attention": true,
  "share_att_key": true,
  "transformers_version": "4.35.2",
  "type_vocab_size": 0,
  "vocab_size": 128100
}

In [21]:
# from transformers import DebertaTokenizer
# tokenizer = DebertaTokenizer.from_pretrained("microsoft/deberta-base")

In [22]:
from transformers import DebertaV2ForSequenceClassification
N_SPLITS = 5
BATCH_SIZE = 64
EPOCHS = 5
SAVE_PATH = 'roberta_model'

kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=0)
test_results = list()

test_dataset = PairsDataset(val.text.values, val.label.values)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, drop_last=False, shuffle=False, collate_fn=data_collator)

for i, (train_index, test_index) in enumerate(kf.split(data)):
    print(f"=====  FOLD {i}  =====")
    train, valid = data.iloc[train_index], data.iloc[test_index]
    train_dataloader, valid_dataloader = get_dataloaders(train, valid)
    
    model = DebertaV2ForSequenceClassification(config).to(device);
    model.train()
    
    train_loop(model, train_dataloader, valid_dataloader, max_epochs=EPOCHS, lr=2e-5, eval_steps = 50)
    
    model.load_state_dict(torch.load(SAVE_PATH))
    model.eval()
    
    _, _, test_probability = evaluate_model(model, test_dataloader)
    test_results.append(test_probability)

=====  FOLD 0  =====
EPOCH 0
step 0 train_loss: 0.702 eval_loss: 0.719 eval_acc: 0.64
step 50 train_loss: 0.656 eval_loss: 0.656 eval_acc: 0.64
step 100 train_loss: 0.653 eval_loss: 0.657 eval_acc: 0.641
EPOCH 1
step 0 train_loss: 0.617 eval_loss: 0.655 eval_acc: 0.64
step 50 train_loss: 0.648 eval_loss: 0.644 eval_acc: 0.633
step 100 train_loss: 0.646 eval_loss: 0.647 eval_acc: 0.636
EPOCH 2
step 0 train_loss: 0.622 eval_loss: 0.649 eval_acc: 0.639
step 50 train_loss: 0.638 eval_loss: 0.644 eval_acc: 0.646
step 100 train_loss: 0.648 eval_loss: 0.648 eval_acc: 0.639
EPOCH 3
step 0 train_loss: 0.597 eval_loss: 0.646 eval_acc: 0.638
step 50 train_loss: 0.624 eval_loss: 0.649 eval_acc: 0.628
step 100 train_loss: 0.633 eval_loss: 0.647 eval_acc: 0.639
EPOCH 4
step 0 train_loss: 0.666 eval_loss: 0.657 eval_acc: 0.612
step 50 train_loss: 0.626 eval_loss: 0.647 eval_acc: 0.642
step 100 train_loss: 0.621 eval_loss: 0.668 eval_acc: 0.623
=====  FOLD 1  =====
EPOCH 0
step 0 train_loss: 0.73 eval

In [25]:
# assert all([len(x) == len(test) for x in test_results])
print(test_results)

predictions = np.mean(test_results, axis = 0)
val.label = [1 if x >= 0.5 else 0 for x in predictions]
val.to_csv('predictions1.tsv', index = False, sep = '\t')

[[0.3428903818130493, 0.29686805605888367, 0.36432650685310364, 0.3688851296901703, 0.41364631056785583, 0.3778071701526642, 0.3678322732448578, 0.2852848768234253, 0.20994886755943298, 0.2908725440502167, 0.4123210310935974, 0.43214505910873413, 0.3379700183868408, 0.46204859018325806, 0.37785500288009644, 0.40267282724380493, 0.27828964591026306, 0.508456289768219, 0.35436195135116577, 0.3758874237537384, 0.4388878643512726, 0.28471603989601135, 0.5225841999053955, 0.42313703894615173, 0.38374975323677063, 0.4060947597026825, 0.45746028423309326, 0.334932416677475, 0.45861420035362244, 0.4873514473438263, 0.32072359323501587, 0.24260541796684265, 0.27572184801101685, 0.28555384278297424, 0.3611781597137451, 0.3891407251358032, 0.40139704942703247, 0.5138797760009766, 0.3052823841571808, 0.5780176520347595, 0.5279096364974976, 0.5063440203666687, 0.29834866523742676, 0.39084023237228394, 0.4266846776008606, 0.5660996437072754, 0.2582542598247528, 0.3041543662548065, 0.4448445439338684

In [27]:
model.load_state_dict(torch.load(SAVE_PATH,map_location=torch.device('cpu')))

<All keys matched successfully>